In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from gensim.models import KeyedVectors
from jieba.analyse import extract_tags, textrank

tf.get_logger().setLevel('ERROR')
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.10.1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from utils.preprocessing import preprocess

raw_ds = preprocess('word2vec')

Total number of news: 30339
Maximum number of news for each group in training set: 64
Minimum number of news for each group in training set: 3
Maximum number of news for each group in test set: 26
Minimum number of news for each group in test set: 3
Maximum number of times a news appears in training set: 22
Minimum number of times a news appears in training set: 0
Maximum number of times a news appears in test set: 11
Minimum number of times a news appears in test set: 0
Loading word2vec model...


In [4]:
cn_model = KeyedVectors.load_word2vec_format(r'D:\fd\semester5\PRML\PJ\assignment2\Chinese-Word-Vectors-master\tencent-ailab-embedding-zh-d100-v0.2.0\tencent-ailab-embedding-zh-d100-v0.2.0.txt', binary=False)

In [5]:
dim = cn_model['中国'].shape[0]
cn_model['中国']

array([ 0.155127, -0.218148, -0.016823, -0.008478, -0.197581,  0.515289,
       -0.527261,  0.271893,  0.392776, -0.155642, -0.038658,  0.452322,
        0.109406, -0.049362, -0.18507 ,  0.335019,  0.06323 , -0.345938,
        0.113851, -0.075246,  0.034878, -0.202701, -0.332777, -0.113046,
       -0.307308,  0.344272,  0.036695,  0.459426, -0.061269, -0.158854,
        0.032959, -0.507191, -0.433246, -0.380269, -0.042545, -0.181901,
       -0.423639, -0.33313 ,  0.034777, -0.242404, -0.170654,  0.279215,
       -0.20025 , -0.257464, -0.184737,  0.145805, -0.302468, -0.007442,
       -0.001378,  0.020027, -0.157774, -0.335372,  0.17455 , -0.483178,
       -0.297272,  0.408632,  0.314054,  0.514542,  0.021476, -0.786786,
       -0.001274,  0.247394,  0.177567,  0.517401,  0.274797,  0.123333,
        0.057318, -0.078237,  0.12561 , -0.092779,  0.794898,  0.549259,
       -0.353153, -0.228939, -0.000885,  0.396531, -0.367841, -0.066524,
       -0.323424,  0.452893,  0.051735,  0.138141, 

In [10]:
mat = {}
titleK, contentK = 1, 4
extra = 5

for ID in raw_ds['news'].keys():
    title, content = raw_ds['news'][ID]
    title_words = extract_tags(title, topK=titleK+extra, withWeight=True)
    content_words = textrank(content, topK=contentK+extra, withWeight=True)

    titleVec = []
    k = 0
    for word, weight in title_words:
        if word in cn_model:
            titleVec.append(cn_model[word]*weight)
            k += 1
        if k == titleK:
            break
    else:
        # pad titleVec to length titleK
        titleVec += [np.zeros(dim)]*(titleK-k)

    contentVec = []
    k = 0
    for word, weight in content_words:
        if word in cn_model:
            contentVec.append(cn_model[word]*weight)
            k += 1
        if k == contentK:
            break
    else:
        # pad contentVec to length contentK
        contentVec += [np.zeros(dim)]*(contentK-k)

    mat[ID] = np.array(titleVec + contentVec)

In [16]:
raw_train = np.array(raw_ds['train'])
raw_valid = np.array(raw_ds['valid'])
raw_test = np.array(raw_ds['test'])
train_label = raw_train[:, 1]
valid_label = raw_valid[:, 1]

C:\Users\Stanl\AppData\Local\Temp\ipykernel_38204\3506262573.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_train = np.array(raw_ds['train'][:split])
C:\Users\Stanl\AppData\Local\Temp\ipykernel_38204\3506262573.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_valid = np.array(raw_ds['train'][split:])


In [19]:
raw_train

array([[list(['44374', '44416', '2913537', '2913541']), 0],
       [list(['3074529', '49422', '49429']), 1],
       [list(['7785735', '132569', '132604', '132610']), 1],
       ...,
       [list(['4825970', '83777', '83780', '4825718']), 0],
       [list(['83546', '4814731', '4816374', '83602', '83640', '4826922']),
        0],
       [list(['170227', '8875478', '170332', '170389', '170423']), 1]],
      dtype=object)

In [30]:
maxlen = 64
train_pad = keras.preprocessing.sequence.pad_sequences(raw_train[:, 0], maxlen=maxlen, padding='post')
valid_pad = keras.preprocessing.sequence.pad_sequences(raw_valid[:, 0], maxlen=maxlen, padding='post')
test_pad = keras.preprocessing.sequence.pad_sequences(raw_ds['test'], maxlen=maxlen, padding='post')
train_pad.shape, valid_pad.shape, test_pad.shape

TypeError: data type 'string' not understood

In [31]:
cn_model.vector_size

100

In [28]:
a = np.vectorize(mat.get)(train_pad[0])
a

array([None, None, None, None,
       array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [25]:
train_data = np.vectorize(mat.get)(train_pad)
valid_data = np.vectorize(mat.get)(valid_pad)
test_data = np.vectorize(mat.get)(test_pad)
train_data.shape, valid_data.shape, test_data.shape

((7158, 64), (796, 64), (3000, 64))

In [26]:
train_data[0]

array([None, None, None, None,
       array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0